In [5]:
import time
import warnings
import inspect
import re
from varname import nameof, argname
import warnings
import json


class MLPath():
   experiments = {}             # dictionary of experiments (e.g, one for each model) that contains a list of logs (runs)
   log = {}                     # dictionary of the current log (run)          
   active = False               # is an MLPath already active
   start_time = None            # to compute the duration of the experiment later 


   @staticmethod
   def start(exp_name):
      assert MLPath.active == False, "You must end the previous experiment before starting a new one."
      MLPath.active = True
      MLPath.log['info'] = {}
      MLPath.log['info']['name'] = exp_name
      MLPath.start_time = time.time()
      MLPath.log['info']['time'] =  time.strftime('%X') 
      MLPath.log['info']['date'] = time.strftime('%x')
   
   @staticmethod
   def clear():
      assert MLPath.active == False, "You must start an experiment before clearing it."
      MLPath.log = {}
      
   
   @staticmethod
   def v(*args, **kwargs):                                     # deprecated
      # uses the syntac func(**v(params)) or func(**v(params, func)) which is not very readable
      if 'func' in kwargs:
         func = kwargs['func']
         del kwargs['func']
         signature = inspect.signature(func)
      else:
         previous_frame = inspect.currentframe().f_back
         text = inspect.getframeinfo(previous_frame)[3][0]        # get the line of code
         text = text.replace(' ', '')                             # remove spaces
         func = re.search(r'\w+(?=\(\*\*l\()', text).group(0)     # get the function name
         exec(f"x = inspect.signature({func})")                   # get the signature of the function
         signature = locals()['x']

      return signature
   
   @staticmethod
   def l(func, name=None):
      # (1, 2, 3, 4, a=5, b=5, 6)
      def wrapped(*args, **kwargs):
         signature = inspect.signature(func)
         
         # Get the parameters of the function
         params = signature.parameters.values()
         
         # the default values of the parameters
         defaults = {param.name: param.default \
            for param in params \
               if param.default != inspect._empty and kwargs.get(param.name) is None}
         
         # will have all the set values of the parameters
         values = {}
         for i, param in enumerate(params):
               # positional arguments not given as keyword arguments must be here
               if i < len(args):                               
                  values[param.name] = args[i]
               # the rest of the parameters are positional arguments given by name or defaults or kwargs
               
         # or are keyword arguments in **kwargs
         values.update(kwargs)
         values.update(defaults)
         
         # Now set the values in the log with the key being the name of the function
         if name:
            MLPath.log[name] = values
         else:
            MLPath.log[func.__name__] = values
         
         return func(*args, **kwargs)
      return wrapped
   
   @staticmethod
   def log_metrics(m1=None, m2=None, m3=None, m4=None, m5=None, m6=None, m7=None, m8=None, m9=None, m10=None, **kwargs):
      MLPath.log['metrics'] = {}
      
      # See if any of m1-m10 are set and if so, add them to the log with the key being the vairable name
      for i in range(1, 11):
         if locals()[f'm{i}'] is not None:
            with warnings.catch_warnings():
               warnings.simplefilter("ignore")           # ignores a useless warning of the varname library
               MLPath.log['metrics'][argname(f'm{i}')] = locals()[f'm{i}']
               
      # Any kwargs are metrics with custom names, add them as well
      MLPath.log['metrics'].update(kwargs)
      
   
   @staticmethod
   def to_log(name, info):
      MLPath.log['other'] = {}
      MLPath.log['other'][name] = info
      
   @staticmethod
   def merge_dicts(dict_list):
    big_dict = {}
    # Get all keys in the top-level, they will be part of the final dict
    keys = set().union(*(d.keys() for d in dict_list))
    for key in keys:
        big_dict[key] = {}
        # Get all possible keys in the 2nd level under the top-level key (if not found, the empty set does nothing to union)
        subkeys = set().union(*(d.get(key, {}) for d in dict_list))
        for subkey in subkeys:
            values = [d.get(key, {}).get(subkey) for d in dict_list]    # None if not found
            big_dict[key][subkey] = values
    return big_dict
   
   
   @staticmethod
   def end():
      duration = time.time() - MLPath.start_time
      # set the duration of the experiment with the appropriate unit
      if duration < 1:
         MLPath.log['info']['duration'] = f'{duration * 1000:.2f} ms'
      elif duration < 60:
         MLPath.log['info']['duration'] = f'{duration:.2f} s'
      elif duration > 60:
         MLPath.log['info']['duration'] = f'{duration / 60:.2f} min'
      elif duration > 3600:
         MLPath.log['info']['duration'] = f'{duration / 3600:.2f} h'
      
      # check if the experiment already exists and set its name and id
      if MLPath.log['info']['name'] in MLPath.experiments:
         exp_name = MLPath.log['info']['name']
         id = len(MLPath.experiments[exp_name]) + 1
         MLPath.log['info']['id'] = id
         MLPath.experiments[exp_name].append(MLPath.log)
      else:
         id = 1
         MLPath.log['info']['id'] = id
         exp_name = MLPath.log['info']['name']
         MLPath.experiments[exp_name] = [MLPath.log]
      MLPath.active = False
      MLPath.log = {}
      
   @staticmethod
   def delete_experiment(exp_name):
      del MLPath.experiments[exp_name]

   @staticmethod
   def delete_run(exp_name, run_id):
      del MLPath.experiments[exp_name][run_id - 1]
      
   def runs_to_json(exp_name):
      runs = MLPath.experiments[exp_name]
      big_dict = MLPath.merge_dicts(runs)
      j = json.dumps(big_dict, indent=3)
      # save the json file
      with open(f'{exp_name}.json', 'w') as f:
         f.write(j)

In [6]:

# let's try this out
l =  MLPath.l
log_metrics = MLPath.log_metrics

def NaiveBayes(alpha, beta_param, c=0, depth_ratio=4, **kwargs):
    return alpha + beta_param + c


def SVMRegressor(x_param, y_param, z_param, **kwargs):
    return x_param * y_param * z_param

def DeepNeuralNet(p_num, k_num, l_num, **kwargs):
    return p_num**k_num + l_num
MLPath.active = False
MLPath.start('NB-SVM-DNN')

accuracy = l(NaiveBayes)(alpha=1024, beta_param=7, c=12,  depth_ratio=538, mega_p=63, g_estim=3, h=43)
gesult = l(SVMRegressor)(14, 510, 4, m_num=63, g_num=3, h_num=4)
result = l(DeepNeuralNet)(12, 2, 3, m_num=62, g_num=3, h_num=4)



MLPath.end()

print(MLPath.experiments['NB-SVM-DNN'][-1])

MLPath.runs_to_json('NB-SVM-DNN')

{'info': {'name': 'NB-SVM-DNN', 'time': '15:38:50', 'date': '02/07/23', 'duration': '0.27 ms', 'id': 1}, 'NaiveBayes': {'alpha': 1024, 'beta_param': 7, 'c': 12, 'depth_ratio': 538, 'mega_p': 63, 'g_estim': 3, 'h': 43}, 'SVMRegressor': {'x_param': 14, 'y_param': 510, 'z_param': 4, 'm_num': 63, 'g_num': 3, 'h_num': 4}, 'DeepNeuralNet': {'p_num': 12, 'k_num': 2, 'l_num': 3, 'm_num': 62, 'g_num': 3, 'h_num': 4}}
